# Algorithm

In [1]:
import numpy as np
from numpy.linalg import LinAlgError, norm
from scipy.linalg import qr, lstsq
from scipy.optimize import OptimizeResult

class _GRGOptimizer:
    """
    Generalized Reduced Gradient (GRG) Optimizer

    Implements:
      - Slack variables for 'ineq' constraints at initialization.
      - Two-phase approach: feasibility, then objective.
      - DFP updates for Hessian after enough feasible steps.
      - A More–Thuente-like line search:
        * Attempt bracket expansion to find an interval [alpha_lo, alpha_hi]
          that presumably contains a local minimizer along the search direction.
        * Once bracketed, call a 'zoom' routine that repeatedly tries a
          cubic (or quadratic) interpolation to pick alpha in (alpha_lo, alpha_hi).
          If alpha is too close to boundaries, we fallback to bisection = mid.
          If we fail to converge after many zoom steps, fallback to minimal step.
        * Enforces interval reduction factor delta in the bracket.
    """

    def __init__(
        self,
        fun,
        x0,
        args=(),
        jac=None,
        constraints=(),
        bounds=None,
        tol=1e-6,
        maxiter=100,
        verbose=False,
        callback=None
    ):
        self.fun = fun
        self.args = args
        self.jac = jac
        self.constraints = constraints
        self.tol = tol
        self.maxiter = maxiter
        self.verbose = verbose
        self.callback = callback

        # separate eq vs ineq constraints
        self.g_eq = []
        self.eq_jacs = []
        self.g_ineq = []
        self.ineq_jacs = []

        for c in constraints:
            ctype = c.get('type', None)
            cfun = c['fun']
            cjac = c.get('jac', None)
            if ctype == 'eq':
                self.g_eq.append(cfun)
                self.eq_jacs.append(cjac)
            elif ctype == 'ineq':
                self.g_ineq.append(cfun)
                self.ineq_jacs.append(cjac)
            else:
                raise ValueError("Constraint must have 'eq' or 'ineq' type.")

        self.n_orig = len(x0)
        if bounds is None:
            bounds = [(None, None)] * self.n_orig
        else:
            if len(bounds) < self.n_orig:
                bounds += [(None, None)]*(self.n_orig - len(bounds))

        x0 = np.array(x0, dtype=float)
        self.m_ineq = len(self.g_ineq)

        # Introduce slack variables for ineq => g_ineq(x)+s_i=0 => s_i >=0
        slack_init = []
        slack_bounds = []
        for i, gf in enumerate(self.g_ineq):
            val = gf(x0, *self.args)
            s_i = max(0.0, -val)
            slack_init.append(s_i)
            slack_bounds.append((0.0, None))

        self.x = np.concatenate([x0, slack_init])
        self.bounds = bounds + slack_bounds
        self.slack_map = {i: self.n_orig + i for i in range(self.m_ineq)}
        self.n = self.n_orig + self.m_ineq

        self.H = None
        self.nfev = 0
        self.njev = 0

        # feasibility
        self.feas_phase = not self.is_feasible(self.x)
        self.feas_iter_count = 0
        self.feasible_steps = 0
        self.dfp_start_steps = 5

        # track line search difficulty
        self.ls_failures = 0

    def _print(self, *args):
        if self.verbose:
            print(*args)

    def call_fun(self, x):
        """ Evaluate objective on original dimension. """
        self.nfev += 1
        return self.fun(x[:self.n_orig], *self.args)

    def call_jac(self, x):
        self.njev += 1
        return self.jac(x[:self.n_orig], *self.args)

    def eval_grad_obj(self, x, eps=1e-8):
        """ If jac is not None => use it, else FD. Slack grad = 0. """
        if self.jac is not None:
            try:
                graw = self.call_jac(x)
                gval = np.zeros(self.n)
                gval[:self.n_orig] = graw
                return gval
            except:
                pass
        f0 = self.call_fun(x)
        gval = np.zeros(self.n)
        for i in range(self.n_orig):
            x_f = x.copy()
            x_f[i]+= eps
            f1 = self.call_fun(x_f)
            gval[i] = (f1 - f0)/eps
        return gval

    def eval_constraints(self, x):
        """ Return eqvals, ineqvals ignoring slack. """
        eqvals = [f(x[:self.n_orig], *self.args) for f in self.g_eq]
        invals= [f(x[:self.n_orig], *self.args) for f in self.g_ineq]
        return eqvals, invals

    def feasibility_violation(self, x):
        eqvals, invals = self.eval_constraints(x)
        eq_viol = sum(abs(v) for v in eqvals)

        bviol=0.0
        for i, (lb, ub) in enumerate(self.bounds):
            val = x[i]
            if lb is not None and val<lb-self.tol:
                bviol+=(lb-val)
            if ub is not None and val>ub+self.tol:
                bviol+=(val-ub)
        return eq_viol + bviol

    def is_feasible(self, x):
        return self.feasibility_violation(x)<= self.tol

    def identify_active_constraints(self, x):
        eqvals, invals= self.eval_constraints(x)
        active=[]
        for i,v in enumerate(eqvals):
            active.append(('eq', i))
        for i,v in enumerate(invals):
            if abs(v)<= self.tol:
                active.append(('ineq', i))
        return active

    def _eval_jac_eq(self, x, eps=1e-8):
        meq = len(self.g_eq)
        J = np.zeros((meq, self.n))
        for i, gf in enumerate(self.g_eq):
            jf = self.eq_jacs[i]
            if jf is not None:
                grad = jf(x[:self.n_orig], *self.args)
                J[i,:self.n_orig]= grad
            else:
                base= gf(x[:self.n_orig], *self.args)
                for jj in range(self.n_orig):
                    xx = x.copy()
                    xx[jj]+= eps
                    f1 = gf(xx[:self.n_orig], *self.args)
                    J[i,jj] = (f1-base)/ eps
        return J

    def _eval_jac_ineq(self, x, eps=1e-8):
        minq = len(self.g_ineq)
        J= np.zeros((minq, self.n))
        for i, gf in enumerate(self.g_ineq):
            jf = self.ineq_jacs[i]
            if jf is not None:
                grad = jf(x[:self.n_orig], *self.args)
                J[i,:self.n_orig] = grad
            else:
                base = gf(x[:self.n_orig], *self.args)
                for jj in range(self.n_orig):
                    xx= x.copy()
                    xx[jj]+= eps
                    f1= gf(xx[:self.n_orig], *self.args)
                    J[i,jj] = (f1-base)/ eps
            # slack
            s_idx = self.slack_map[i]
            J[i, s_idx] = 1.0
        return J

    def form_jacobian_active(self, x, active):
        eqJ= self._eval_jac_eq(x)
        inJ= self._eval_jac_ineq(x)
        m= len(active)
        A= np.zeros((m, self.n))
        row=0
        for (ctype, i) in active:
            if ctype=='eq':
                A[row,:] = eqJ[i,:]
            else:
                A[row,:] = inJ[i,:]
            row+=1
        return A

    def is_bound_active(self, x, idx):
        lb, ub= self.bounds[idx]
        val = x[idx]
        if lb is not None and abs(val-lb)< self.tol:
            return True
        if ub is not None and abs(val-ub)< self.tol:
            return True
        return False

    def partition_variables(self, x, active, J):
        """
        rank reveal => dep, indep. Avoid bound-active if possible
        """
        if J.size==0 or len(active)==0:
            return np.array([],dtype=int), np.arange(self.n, dtype=int)
        Q, R, pivot = qr(J, mode='economic', pivoting=True)
        rank = np.linalg.matrix_rank(J)
        dep_i = pivot[:rank]
        for i, di in enumerate(dep_i):
            if self.is_bound_active(x, di):
                candidate=None
                for c in pivot[rank:]:
                    if not self.is_bound_active(x, c):
                        candidate=c
                        break
                if candidate is not None:
                    dep_i[i] = candidate
        dep_i= np.unique(dep_i)
        all_i= np.arange(self.n)
        indep_i= np.array([ii for ii in all_i if ii not in dep_i])
        return dep_i, indep_i

    def compute_reduced_gradient(self, x, active, J):
        gf = self.eval_grad_obj(x)
        if len(active)==0:
            return gf, gf
        JT= J.T
        try:
            lam, _, _, _= lstsq(JT, -gf, lapack_driver='gelsy')
        except LinAlgError:
            lam= np.zeros(JT.shape[1])
        red = gf + JT@ lam
        return red, gf

    def newton_raphson(self, x, active, dep_indices, max_iter=50):
        """
        Solve constraints => B dy=-r. Then bound fix.
        Return (x, success).
        """
        for it in range(max_iter):
            eqvals, invals= self.eval_constraints(x)
            res=[]
            for (ctype, i) in active:
                if ctype=='eq':
                    val= self.g_eq[i](x[:self.n_orig], *self.args)
                    res.append(val)
                else:
                    s_idx= self.slack_map[i]
                    val= self.g_ineq[i](x[:self.n_orig], *self.args)+ x[s_idx]
                    res.append(val)
            rr= np.array(res)
            if norm(rr)< self.tol:
                return x, True
            J= self.form_jacobian_active(x, active)
            if J.size==0:
                return x, True
            B= J[:,dep_indices]
            try:
                dy, _, rnk, _= lstsq(B, -rr, lapack_driver='gelsy')
                if rnk< len(dep_indices):
                    self._print("[DEBUG] NewtonR => rank deficiency.")
                    return x, False
                x[dep_indices]+= dy
            except LinAlgError:
                self._print("[DEBUG] NewtonR => LinAlg error.")
                return x, False
            # bound fix
            for ii,(lb, ub) in enumerate(self.bounds):
                if lb is not None and x[ii]< lb:
                    x[ii]= lb
                if ub is not None and x[ii]> ub:
                    x[ii]= ub
        self._print("[DEBUG] NewtonR => no converge => max_iter")
        return x, False

    def dfp_update(self, H, s, y):
        sy= s@y
        yHy= y @H @y
        if abs(sy)<1e-14 or abs(yHy)<1e-14:
            self._print("[DEBUG] skip DFP => small denom.")
            return H
        s_sT= np.outer(s, s)
        Hy= H@ y
        return H + (s_sT/sy) - (np.outer(Hy,Hy)/yHy)

    def minimal_step_attempt(self, x, d, active, dep, indep):
        alpha=1e-6
        xx= x.copy()
        xx[indep]+= alpha*d[indep]
        xx,nok= self.newton_raphson(xx, active, dep)
        if nok:
            oldf= self.call_fun(x)
            newf= self.call_fun(xx)
            if self.is_feasible(xx) or newf< oldf:
                self._print("[DEBUG] minimal step => success alpha=", alpha)
                return alpha, 'success', xx
        self._print("[DEBUG] minimal step => no improve.")
        return alpha, 'no_improve', x

    def _compute_feas_dir(self, x):
        eqvals, invals= self.eval_constraints(x)
        grad_viol= np.zeros(self.n)
        eqJ= self._eval_jac_eq(x)
        for i,val in enumerate(eqvals):
            if abs(val)> self.tol:
                sgn= 1.0 if val>0 else -1.0
                grad_viol[:self.n_orig]+= sgn* eqJ[i,:self.n_orig]
        # bound
        for i,(lb, ub) in enumerate(self.bounds):
            val= x[i]
            if lb is not None and val<(lb-self.tol):
                grad_viol[i]-=1.0
            if ub is not None and val>(ub+self.tol):
                grad_viol[i]+=1.0
        if norm(grad_viol)<1e-14:
            return -grad_viol
        return -grad_viol

    def _cubic_interpolate(self, al, fl, dl, ah, fh, dh):
        """
        Standard More–Thuente style cubic interpolation.
        clamp alpha in [0.1*al+0.9*ah, 0.9*al+0.1*ah].
        """
        d1= dl+ dh - 3*(fl- fh)/(al- ah)
        d2sq= d1*d1 - dl*dh
        if d2sq<0:
            self._print("[DEBUG] cubic => negative discriminant => mid.")
            return 0.5*(al+ ah)
        d2= np.sqrt(d2sq)
        # sign depends on al<ah or not
        if ah< al:
            d2= -d2
        alpha_c = ah - ( (ah-al)*( dh+ d2 - d1 ) / (dh- dl+ 2*d2 ) )
        # safeguard
        lower= 0.1* al + 0.9* ah
        upper= 0.9* al + 0.1* ah
        if lower> upper:
            lower, upper= upper, lower
        alpha_c = max(min(alpha_c, upper), lower)
        self._print(f"[DEBUG] cubic => alpha_c={alpha_c}")
        return alpha_c

    def _bisect(self, al, ah):
        return 0.5*(al+ ah)

    def line_search(self, x, d, active, dep, indep):
        """
        Attempt bracket then zoom. If repeated fails => minimal step fallback.
        """
        tries= 2
        for attempt in range(1, tries+1):
            alpha, st, xnew= self._mt_search(x, d, active, dep, indep)
            self._print(f"[DEBUG] line_search attempt={attempt}, status={st}, alpha={alpha}")
            if st=='success':
                return alpha, st, xnew
            if st in ['nr_fail','not_descent','max_ls_iter']:
                return alpha, st, xnew
            # st='no_improve' => repeat
        # fallback => minimal
        alpha_ms, st_ms, x_ms= self.minimal_step_attempt(x, d, active, dep, indep)
        return alpha_ms, st_ms, x_ms

    def _mt_search(self, x, d, active, dep, indep):
        """
        Entire More–Thuente: bracket => zoom.
        We'll set c1=1e-4, c2=0.9, xtrapf=2.0 expansions
        delta=0.66 for forced bisection
        """
        c1=1e-4
        c2=0.9
        delta=0.66
        xtrapf=2.0
        f0= self.call_fun(x)
        g0= self.eval_grad_obj(x)
        dd0= g0@ d
        if dd0>=0:
            self._print("[DEBUG] not descent => dd0=", dd0)
            return 0.0, 'not_descent', x

        # Try alpha=1 => if NR fail => keep halving
        alpha=1.0
        fa, dda, xa= self._eval_alpha(x, alpha, d, active, dep, indep)
        if fa is None:
            for rep in range(5):
                alpha*=0.5
                fa, dda, xa= self._eval_alpha(x, alpha, d, active, dep, indep)
                if fa is not None:
                    break
            else:
                return alpha, 'nr_fail', x
        if fa is None:
            return alpha, 'nr_fail', x

        # check quick success
        if fa<= f0 + c1*alpha*dd0 and abs(dda)<= c2*abs(dd0):
            self._print(f"[DEBUG] immediate strong wolfe => alpha={alpha}")
            return alpha, 'success', xa

        # bracket expand
        alpha_lo= 0.0
        f_lo= f0
        dd_lo= dd0
        alpha_hi= None
        f_hi=None
        dd_hi=None

        prev_alpha= alpha_lo
        prev_f= f_lo
        prev_dd= dd_lo
        max_br= 10
        self._print("[DEBUG] bracket expansion phase.")
        for ib in range(1, max_br+1):
            # check conditions
            if (fa> f0 + c1*alpha*dd0) or (ib>1 and fa>= prev_f):
                alpha_hi= alpha
                f_hi= fa
                dd_hi= dda
                break
            if abs(dda)<= c2*abs(dd0):
                return alpha, 'success', xa
            if dda>=0:
                alpha_hi= alpha
                f_hi= fa
                dd_hi= dda
                break
            # else update and expand
            prev_alpha= alpha
            prev_f= fa
            prev_dd= dda
            alpha*= xtrapf
            fa, dda, xa= self._eval_alpha(x, alpha, d, active, dep, indep)
            self._print(f"[DEBUG] bracket iter={ib}, alpha={alpha}, f={fa}, dd={dda}")
            if fa is None:
                return alpha, 'nr_fail', x
        else:
            # never bracket => no_improve
            self._print("[DEBUG] bracket => no bracket => no_improve.")
            return alpha, 'no_improve', xa

        if alpha_hi is None:
            # no bracket => no_improve
            return alpha, 'no_improve', xa

        if ib==1 and alpha_hi== alpha:
            # set lo => previous
            alpha_lo= prev_alpha
            f_lo= prev_f
            dd_lo= prev_dd
        else:
            alpha_lo= prev_alpha
            f_lo= prev_f
            dd_lo= prev_dd

        # now zoom
        return self._zoom(x, d, active, dep, indep,
                          alpha_lo, alpha_hi,
                          f_lo, f_hi,
                          dd_lo, dd_hi,
                          f0, dd0,
                          c1, c2,
                          delta)

    def _zoom(self, x, d, active, dep, indep,
              alpha_lo, alpha_hi,
              f_lo, f_hi,
              dd_lo, dd_hi,
              f0, dd0,
              c1, c2,
              delta):
        max_zoom=15
        old_width= abs(alpha_hi- alpha_lo)
        self._print(f"[DEBUG] zoom => [lo={alpha_lo}, hi={alpha_hi}]")
        for iz in range(1, max_zoom+1):
            width= abs(alpha_hi- alpha_lo)
            if width<= 1e-14:
                self._print("[DEBUG] zoom => bracket collapsed => no_improve.")
                return alpha_lo, 'no_improve', x
            # if width >= delta*old_width => do bisection
            if width>= delta* old_width:
                alpha_j= self._bisect(alpha_lo, alpha_hi)
                self._print("[DEBUG] no sufficient interval shrink => bisection => alpha_j=", alpha_j)
            else:
                # do cubic
                alpha_j= self._cubic_interpolate(alpha_lo, f_lo, dd_lo, alpha_hi, f_hi, dd_hi)
                # check if too close => bisection
                eps_small= 1e-6
                if (abs(alpha_j-alpha_lo)< eps_small*width) or (abs(alpha_j-alpha_hi)< eps_small*width):
                    self._print("[DEBUG] alpha too close => bisection.")
                    alpha_j= self._bisect(alpha_lo, alpha_hi)

            old_width= width
            fj, ddj, xj= self._eval_alpha(x, alpha_j, d, active, dep, indep)
            self._print(f"[DEBUG] zoom iter={iz}, alpha_j={alpha_j}, f={fj}, dd={ddj}")

            if fj is None:
                # fallback => half alpha_j
                alpha_j*=0.5
                fj, ddj, xj= self._eval_alpha(x, alpha_j, d, active, dep, indep)
                if fj is None:
                    self._print("[DEBUG] zoom => repeated NR fail => stop.")
                    return alpha_j, 'nr_fail', x

            # Armijo check
            if (fj> f0 + c1* alpha_j*dd0) or (fj>= f_lo):
                alpha_hi= alpha_j
                f_hi= fj
                dd_hi= ddj
            else:
                # check curvature
                if abs(ddj)<= c2* abs(dd0):
                    self._print(f"[DEBUG] zoom => success => alpha={alpha_j}")
                    return alpha_j, 'success', xj
                if ddj*(alpha_hi - alpha_lo)>=0:
                    alpha_hi= alpha_lo
                    f_hi= f_lo
                    dd_hi= dd_lo
                alpha_lo= alpha_j
                f_lo= fj
                dd_lo= ddj
        self._print("[DEBUG] zoom => max_zoom => no_improve.")
        return alpha_lo, 'no_improve', x

    def _eval_alpha(self, x, alpha, d, active, dep, indep):
        """
        x + alpha d => newton => feasible? => compute f, dd.
        If newton fail => None
        """
        x_trial= x.copy()
        x_trial[indep]+= alpha*d[indep]
        x_nr, ok= self.newton_raphson(x_trial, active, dep)
        if not ok:
            return None, None, x_trial
        fv= self.call_fun(x_nr)
        gv= self.eval_grad_obj(x_nr)
        dd= gv@ d
        return fv, dd, x_nr

    def check_kkt(self, red_grad):
        return norm(red_grad)< self.tol

    def run(self):
        x= self.x
        self.H= np.eye(self.n)
        success=False
        message=""

        for it in range(1, self.maxiter+1):
            self._print(f"\n--- Iteration {it} ---")

            if self.feas_phase:
                self.feas_iter_count+=1
                if self.feas_iter_count>50:
                    message="Feasibility not met in 50 steps."
                    success=False
                    break

            active= self.identify_active_constraints(x)
            J= self.form_jacobian_active(x, active)
            dep, indep= self.partition_variables(x, active, J)

            if self.feas_phase and self.is_feasible(x):
                self.feas_phase= False
                self.feasible_steps=0
                self.H= np.eye(self.n)
                self._print("[DEBUG] transition => objective phase")

            if not self.feas_phase:
                redg, gf= self.compute_reduced_gradient(x, active, J)
                rg_n= norm(redg)
                self._print(f"[DEBUG] reduced grad norm={rg_n}")
                if self.check_kkt(redg):
                    message= "KKT => done"
                    success=True
                    break
                use_dfp= (self.feasible_steps>= self.dfp_start_steps)
                if use_dfp:
                    if self.H is None or self.H.shape[0]!= self.n:
                        self.H= np.eye(self.n)
                    d= -self.H@ redg
                    self._print("[DEBUG] using DFP direction.")
                else:
                    d= -gf
                    self._print("[DEBUG] using steepest direction.")
            else:
                self._print("[DEBUG] feasibility => reduce violation.")
                d= self._compute_feas_dir(x)
                success=False
                message=""

            alpha, st, xnew= self.line_search(x, d, active, dep, indep)
            self._print(f"[DEBUG] line search => alpha={alpha}, status={st}")
            if st in ['not_descent','nr_fail','no_improve','max_ls_iter']:
                message= f"line search => {st}"
                success=False
                break

            if not self.feas_phase:
                if self.is_feasible(xnew):
                    self.feasible_steps+=1
                if self.feasible_steps>= self.dfp_start_steps:
                    s= xnew- x
                    gf_new= self.eval_grad_obj(xnew)
                    y= gf_new- gf
                    self.H= self.dfp_update(self.H, s, y)

            x= xnew
            if self.callback is not None:
                self.callback(x)
        else:
            message= "Max iteration."
            success=False

        xfinal= x[:self.n_orig]
        res= OptimizeResult()
        res.x= xfinal
        res.fun= self.call_fun(x)
        fullg= self.eval_grad_obj(x)
        res.jac= fullg[:self.n_orig]
        res.hess_inv= self.H[:self.n_orig,:self.n_orig]
        res.nit= it
        res.nfev= self.nfev
        res.njev= self.njev
        res.success= success
        res.message= message
        return res

# Interface

In [2]:
def grg_minimize(
    fun,
    x0,
    args=(),
    jac=None,
    constraints=(),
    bounds=None,
    tol=1e-6,
    options=None,
    callback=None,
    verbose=False
):
    """
    A SciPy-like interface to the GRG optimizer with a More–Thuente-inspired line search.

    Parameters
    ----------
    fun : callable
        Objective function: fun(x, *args)-> float
    x0 : array-like
        Initial guess.
    args : tuple
        Additional arguments.
    jac : callable or None
        Gradient or None => finite diff.
    constraints : list of dict
        'type': 'eq' or 'ineq', 'fun', possibly 'jac'.
    bounds : list of (float, float)
        For each original var.
    tol : float
        Tolerance for feasibility & KKT.
    options : dict
        e.g. {'maxiter':100}
    callback : callable
        Called each iteration
    verbose : bool
        Print debug info

    Returns
    -------
    OptimizeResult
        With x, fun, jac, hess_inv, nit, nfev, njev, success, message
    """
    if options is None:
        options={}
    maxiter= options.get('maxiter',100)

    optimizer= _GRGOptimizer(
        fun=fun, x0=x0, args=args,
        jac=jac, constraints=constraints,
        bounds=bounds,
        tol=tol, maxiter=maxiter,
        verbose=verbose, callback=callback
    )
    return optimizer.run()

# Test Cases

In [3]:
# Test 1: Simple Unconstrained Quadratic
# Minimize f(x) = x^2
# Should converge easily to x = 0.

def fun1(x, *args):
    return x[0]**2

def grad1(x, *args):
    return np.array([2 * x[0]])

x0_1 = np.array([2.0])  # start from 2

res1 = grg_minimize(fun1, x0_1, jac=grad1, constraints=[], bounds=None, tol=1e-8, verbose=True)
print("Test 1 Result:", res1)


--- Iteration 1 ---
[DEBUG] reduced grad norm=4.0
[DEBUG] using steepest direction.
[DEBUG] bracket expansion phase.
[DEBUG] zoom => [lo=0.0, hi=1.0]
[DEBUG] no sufficient interval shrink => bisection => alpha_j= 0.5
[DEBUG] zoom iter=1, alpha_j=0.5, f=0.0, dd=0.0
[DEBUG] zoom => success => alpha=0.5
[DEBUG] line_search attempt=1, status=success, alpha=0.5
[DEBUG] line search => alpha=0.5, status=success

--- Iteration 2 ---
[DEBUG] reduced grad norm=0.0
Test 1 Result:   message: KKT => done
  success: True
      fun: 0.0
        x: [ 0.000e+00]
      nit: 2
      jac: [ 0.000e+00]
 hess_inv: [[ 1.000e+00]]
     nfev: 4
     njev: 6


In [4]:
# Test 2: Unconstrained 2D Quadratic
# Minimize f(x, y) = (x - 1)^2 + (y + 2)^2
# Known minimum at (1, -2).

def fun2(x, *args):
    return (x[0] - 1)**2 + (x[1] + 2)**2

def grad2(x, *args):
    return np.array([2 * (x[0] - 1), 2 * (x[1] + 2)])

x0_2 = np.array([0.0, 0.0]) # start from (0, 0)

res2 = grg_minimize(fun2, x0_2, jac=grad2, constraints=[], bounds=None, tol=1e-8, verbose=True)
print("Test 2 Result:", res2)


--- Iteration 1 ---
[DEBUG] reduced grad norm=4.47213595499958
[DEBUG] using steepest direction.
[DEBUG] bracket expansion phase.
[DEBUG] zoom => [lo=0.0, hi=1.0]
[DEBUG] no sufficient interval shrink => bisection => alpha_j= 0.5
[DEBUG] zoom iter=1, alpha_j=0.5, f=0.0, dd=0.0
[DEBUG] zoom => success => alpha=0.5
[DEBUG] line_search attempt=1, status=success, alpha=0.5
[DEBUG] line search => alpha=0.5, status=success

--- Iteration 2 ---
[DEBUG] reduced grad norm=0.0
Test 2 Result:   message: KKT => done
  success: True
      fun: 0.0
        x: [ 1.000e+00 -2.000e+00]
      nit: 2
      jac: [ 0.000e+00  0.000e+00]
 hess_inv: [[ 1.000e+00  0.000e+00]
            [ 0.000e+00  1.000e+00]]
     nfev: 4
     njev: 6


In [5]:
# Test 3: Problem with an Equality Constraint
# Minimize f(x, y) = x^2 + y^2 subject to x + y = 1 (an equality constraint).
# The solution should lie on the line y = 1 - x. The minimum is at (0.5, 0.5).

def fun3(x, *args):
    return x[0]**2 + x[1]**2

def grad3(x, *args):
    return np.array([2*x[0], 2*x[1]])

def eq_constr3(x, *args):
    return x[0] + x[1] - 1.0

# Constraint dict format: {'type' : 'eq', 'fun' : ..., 'jac' : ...}
def eq_jac3(x, *args):
    return np.array([1.0, 1.0])

constraints_3 = [{'type' : 'eq', 'fun' : eq_constr3, 'jac' : eq_jac3}]
x0_3 = np.array([2.0, -1.0]) # start from a point not on the line

res3 = grg_minimize(fun3, x0_3, jac=grad3, constraints=constraints_3, bounds=None, tol=1e-8, verbose=True)
print("Test 3 Result:", res3)


--- Iteration 1 ---
[DEBUG] reduced grad norm=4.242640687119285
[DEBUG] using steepest direction.
[DEBUG] immediate strong wolfe => alpha=1.0
[DEBUG] line_search attempt=1, status=success, alpha=1.0
[DEBUG] line search => alpha=1.0, status=success

--- Iteration 2 ---
[DEBUG] reduced grad norm=1.4142135623730951
[DEBUG] using steepest direction.
[DEBUG] bracket expansion phase.
[DEBUG] zoom => [lo=0.0, hi=1.0]
[DEBUG] no sufficient interval shrink => bisection => alpha_j= 0.5
[DEBUG] zoom iter=1, alpha_j=0.5, f=1.0, dd=0.0
[DEBUG] cubic => alpha_c=0.16666666666666669
[DEBUG] zoom iter=2, alpha_j=0.16666666666666669, f=0.5555555555555556, dd=-2.6666666666666665
[DEBUG] zoom => success => alpha=0.16666666666666669
[DEBUG] line_search attempt=1, status=success, alpha=0.16666666666666669
[DEBUG] line search => alpha=0.16666666666666669, status=success

--- Iteration 3 ---
[DEBUG] reduced grad norm=0.47140452079103157
[DEBUG] using steepest direction.
[DEBUG] bracket expansion phase.
[DEBU

In [6]:
# Test 4: Inequality Constraint
# Minimize f(x) = x^2 subject to x ≥ 0 (inequality).
# Start from x = -2. Must move into feasible region and then minimize at x = 0.

def fun4(x, *args):
    return x[0]**2

def grad4(x, *args):
    return np.array([2 * x[0]])

def ineq_constr4(x, *args):
    # x ≥ 0 means -x ≤ 0
    return -x[0]

constraints_4 = [{'type' : 'ineq', 'fun' : ineq_constr4}]
x0_4 = np.array([-2.0])

res4 = grg_minimize(fun4, x0_4, jac=grad4, constraints=constraints_4, bounds=None, tol=1e-8, verbose=True)
print("Test 4 Result:", res4)


--- Iteration 1 ---
[DEBUG] reduced grad norm=4.0
[DEBUG] using steepest direction.
[DEBUG] bracket expansion phase.
[DEBUG] zoom => [lo=0.0, hi=1.0]
[DEBUG] no sufficient interval shrink => bisection => alpha_j= 0.5
[DEBUG] zoom iter=1, alpha_j=0.5, f=0.0, dd=0.0
[DEBUG] zoom => success => alpha=0.5
[DEBUG] line_search attempt=1, status=success, alpha=0.5
[DEBUG] line search => alpha=0.5, status=success

--- Iteration 2 ---
[DEBUG] reduced grad norm=0.0
Test 4 Result:   message: KKT => done
  success: True
      fun: 0.0
        x: [ 0.000e+00]
      nit: 2
      jac: [ 0.000e+00]
 hess_inv: [[ 1.000e+00]]
     nfev: 4
     njev: 6


In [7]:
# Test 5: Problem with both bounds and constraints
# Minimize f(x, y) = (x - 2)^2 + (y - 3)^2 with bound x ≥ 1 and eq constraint: x + y =5
# The solution: from x + y = 5, if we want to minimize distance to (2, 3),
# The unconstrained min would be (2, 3). This satisfies x ≥ 1. So solution should be (2, 3).

def fun5(x, *args):
    return (x[0] - 2)**2 + (x[1] - 3)**2

def grad5(x, *args):
    return np.array([2 * (x[0] - 2), 2 * (x[1] - 3)])

def eq_constr5(x, *args):
    return x[0] + x[1] - 5.0

def eq_jac5(x, *args):
    return np.array([1.0, 1.0])

constraints_5 = [{'type' : 'eq', 'fun' : eq_constr5, 'jac' : eq_jac5}]
bounds_5 = [(1.0, None), (None, None)] # x ≥ 1

x0_5 = np.array([1.5, 4.0]) # start from a feasible point close to solution
res5 = grg_minimize(fun5, x0_5, jac=grad5, constraints=constraints_5, bounds=bounds_5, tol=1e-8, verbose=True)
print("Test 5 Result:", res5)


--- Iteration 1 ---
[DEBUG] feasibility => reduce violation.
[DEBUG] immediate strong wolfe => alpha=1.0
[DEBUG] line_search attempt=1, status=success, alpha=1.0
[DEBUG] line search => alpha=1.0, status=success

--- Iteration 2 ---
[DEBUG] transition => objective phase
[DEBUG] reduced grad norm=0.0
Test 5 Result:   message: KKT => done
  success: True
      fun: 0.0
        x: [ 2.000e+00  3.000e+00]
      nit: 2
      jac: [ 0.000e+00  0.000e+00]
 hess_inv: [[ 1.000e+00  0.000e+00]
            [ 0.000e+00  1.000e+00]]
     nfev: 3
     njev: 4


In [8]:
# Test 6: Flat Region (Potential line search trouble)
# Minimize f(x) = |x|^3, which is flat at x = 0. A weird shape that can cause line search to try multiple steps.
# The minimum is at x = 0. Start from x = 10. Check if line search can handle flatness near zero.

def fun6(x, *args):
    return abs(x[0])**3

def grad6(x, *args):
    # Grad of |x|^3 = 3|x|^2 * sign(x) but handle x = 0:
    val = x[0]
    if val > 0:
        return np.array([3 * val**2])
    elif val < 0:
        return np.array([-3 * val**2])
    else:
        # Non-differentiable at zero, approximate gradient as 0 for code simplicity
        return np.array([0.0])

x0_6 = np.array([10.0])
res6 = grg_minimize(fun6, x0_6, jac=grad6, constraints=[], bounds=None, tol=1e-8, verbose=True)
print("Test 6 Result:", res6)


--- Iteration 1 ---
[DEBUG] reduced grad norm=300.0
[DEBUG] using steepest direction.
[DEBUG] bracket expansion phase.
[DEBUG] zoom => [lo=0.0, hi=1.0]
[DEBUG] no sufficient interval shrink => bisection => alpha_j= 0.5
[DEBUG] zoom iter=1, alpha_j=0.5, f=2744000.0, dd=17640000.0
[DEBUG] cubic => alpha_c=0.06763333313741393
[DEBUG] zoom iter=2, alpha_j=0.06763333313741393, f=1089.5473703297262, dd=95295.68891135429
[DEBUG] cubic => alpha_c=0.03357496292351557
[DEBUG] zoom iter=3, alpha_j=0.03357496292351557, f=0.0003809027569635412, dd=4.729173566982428
[DEBUG] zoom => success => alpha=0.03357496292351557
[DEBUG] line_search attempt=1, status=success, alpha=0.03357496292351557
[DEBUG] line search => alpha=0.03357496292351557, status=success

--- Iteration 2 ---
[DEBUG] reduced grad norm=0.015763911889941428
[DEBUG] using steepest direction.
[DEBUG] immediate strong wolfe => alpha=1.0
[DEBUG] line_search attempt=1, status=success, alpha=1.0
[DEBUG] line search => alpha=1.0, status=succe